In [3]:
import ipywidgets as widgets
import urllib.request

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle,
    GeoJSON,
    DrawControl,
    FullScreenControl,
    WidgetControl
)
zoom_default = 3
center_default=[40, -100]
m = Map(zoom = zoom_default, center = center_default)

In [4]:
m.clear_layers()


def clear_m():
    global polys, circles, files_list
    polys = set()
    circles = set()
    files_list = []


clear_m()
m.clear_controls()

rect_color = '#a52a2a'
poly_color = '#00F'
circle_color = '#84f59b'

zoom = m.zoom
center = m.center

print(zoom)
print(center)

m = Map(zoom=zoom,center=center,scroll_wheel_zoom=True)

myDrawControl = DrawControl(
        rectangle={'shapeOptions':{'color':rect_color}},
        polygon={'shapeOptions':{'color':poly_color}},
        circle={'shapeOptions':{'color':circle_color}})


layout = widgets.Layout(width='auto')

def download_all(urls):
    if urls != []:
        for url in file_urls:            
            local_file = 'E:/Lidar DataSet/' + '/'.join(url.split('/')[-2:])
            #print(local_file)
            try:
                if not os.path.exists(local_file):
                    print('start downloading ' + local_file)
                    #request.urlretrieve(url, local_file)
                else:
                    print(local_file + ' exists')
            except Exception as e:
                print(e)
                continue
    else:
        print("there is no downloading source!")

def get_result(files_list):
    results = []
    for file_urls in files_list:
        #print(file_urls)
        for url in file_urls:
            #print(url)
            if url not in results:
                results.append(url)               
    return results

#get 
import requests
import json
import urllib
import os
from urllib import request
import pandas as pd
from IPython.display import display

post_polygon = 'http://206.12.92.18:10085/lidar/polygon'
post_circle = 'http://206.12.92.18:10085/lidar/circle'

def get_circle(circles):
    for r in circles:
        r = list(r)
        pload = post_circle + "?longitude=" + str(r[0]) + "&latitude=" + str(r[1]) + "&radius=" + str(round(r[2]))
        r = requests.get(pload)
        #print(r.json()['total_num'])
        files = r.json()['regions']
        #print(files)
        return files

def get_polys(polys):

    for r in polys: 
        vertices = []
        #print("\nPolygon vertices: ") 
        for c in r: 
          #print('%f, %f' % c)
          temp = [c[0], c[1]]
          vertices.append(temp)

        vertices.append(vertices[0])
        pload = { 'vertices': vertices}
        r = requests.post(post_polygon, data = json.dumps(pload), headers={"Content-Type":"application/json"})
        #print(r.json()['total_num'])
        files = r.json()['regions']
        #print(files)
        return files

def handle_draw(self, action, geo_json):
    if geo_json['geometry']['type'] == 'Polygon' or geo_json['geometry']['type'] == 'Rectangle':
        global polys
        polygon=[]
        for coords in geo_json['geometry']['coordinates'][0][:-1][:]:
            polygon.append(tuple(coords))
        polygon = tuple(polygon)     
        if action == 'created':
            print("action created")
            polys.add(polygon)
        elif action == 'deleted':
            print("action deleted")
            polys.discard(polygon)
        files_list.append(get_polys(polys))
    if geo_json['geometry']['type'] == 'Point' and geo_json['properties']['style']['radius'] != None:
        global circles
        center = geo_json['geometry']['coordinates']
        radius = geo_json['properties']['style']['radius']
        circle = tuple(tuple([center[0], center[1], radius]))
        if action == 'created':
            circles.add(circle)
        elif action == 'deleted':
            circles.discard(circle)
        files_list.append(get_circle(circles))
        
        
## add 
        
# # Add a button to the map
button = widgets.Button(description="Clear drawings")
btn_control = WidgetControl(widget=button, position='bottomleft')
m.add_control(btn_control)

# # Handle click event
def on_button_clicked(b):
    myDrawControl.clear()
    clear_m()

button.on_click(on_button_clicked)

button2 = widgets.Button(description="Show Result")
btn_control2 = WidgetControl(widget=button2, position='topright')
m.add_control(btn_control2)

dropdown = widgets.Dropdown(
    options= get_result(files_list),
    description='URLs:',
    disabled=False,
    layout = layout
    )
dw_dropdown = WidgetControl(widget=dropdown, position='topright')
m.add_control(dw_dropdown)


def on_button_clicked2(b):
    dw_dropdown.widget.options = get_result(files_list)

button2.on_click(on_button_clicked2)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new']) 
        btn_control4.widget.description = "download:"+change['new']
        

dropdown.observe(on_change)


button3 = widgets.Button(description="Download All")
btn_control3 = WidgetControl(widget=button3, position='bottomright')
m.add_control(btn_control3)

def on_button_clicked3(b):
    #download all 
    print(get_result(files_list))
    download_all(get_result(files_list))

button3.on_click(on_button_clicked3)

button4 = widgets.Button(description="Single Download",
                        layout = layout)
btn_control4 = WidgetControl(widget=button4, position='bottomright')
m.add_control(btn_control4)

def on_button_clicked4(b):
    #download all 
    if btn_control4.widget.description == "" or btn_control4.widget.description == "Single Download":
        print("no file choose!")
    else:
        url = btn_control4.widget.description.removeprefix('download:')
        local_file = 'E:/Lidar DataSet/' + '/'.join(url.split('/')[-2:])
        try:
            if not os.path.exists(local_file):
                print('start downloading ' + local_file)
                request.urlretrieve(url, local_file)
            else:
                print(local_file + ' exists')
        except Exception as e:
            print(e)

button4.on_click(on_button_clicked4)

myDrawControl.on_draw(handle_draw)
m.add_control(myDrawControl)
m.add_control(FullScreenControl())

m


3.0
[40, -100]


Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

action created
changed to http://206.12.92.18:10083/lidar/northvancouver/bc_092g035_2_1_2_xyes_8_utm10_20170713.laz
